In [2]:
import pandas as pd
import random
from tqdm.notebook import tqdm

### 원본데이터 : 범죄종류_연도별_자치구별_범죄건수

In [3]:
perYear=pd.read_csv('./절도.csv')
perYear=perYear.iloc[:,1:]
perYear.columns=['자치구', '2014', '2015', '2016', '2017', '2018', '2019', '2020','2021']
perYear.head(3)

,자치구,2014,2015,2016,2017,2018,2019,2020,2021
0,종로구,2272,2184,1940,1644,1483,1515,1183,1079
1,중구,2576,2548,2150,1846,1855,2202,1642,1279
2,용산구,1560,1557,1489,1346,1096,999,1111,945


### 범죄종류_연도별_자치구별_범죄건수  => 범죄종류_월별_자치구별_범죄건수
- 월별로 나눈 빈 데이터프레임 생성

In [8]:
perMonth=pd.DataFrame(index=perYear.index,columns=[f'20{j}.{i}' for j in range(14,22) for i in range(1, 13)])
perMonth.head(3)

,2014.1,2014.2,2014.3,2014.4,2014.5,2014.6,2014.7,2014.8,2014.9,2014.10,...,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11,2021.12
자치구,,,,,,,,,,,,,,,,,,,,,
종로구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
중구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
용산구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 범죄종류_월별_가중치 생성

In [9]:
weight=pd.read_csv('./절도_월별_건수.csv')
weight=weight.iloc[1:,3:]
weight=weight.astype('int32')
weight.head(3)

,2014,2014.1,2014.2,2014.3,2014.4,2014.5,2014.6,2014.7,2014.8,2014.9,...,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11,2021.12
1,266784,21299,19044,22751,22059,24550,24278,25363,24259,23934,...,14260,15064,14768,15814,15507,14868,14651,14562,12852,11220


In [10]:
for i in range(8):
    for j in range(12):
        # i번째 년 j번째 달의 범죄 건수 / i번째 년 범죄 건수 합계  -> 비율 생성
        weight.iloc[0,13*i+j+1]=weight.iloc[0,13*i+j+1]/weight.iloc[0,13*i]
weight.drop(columns=['2014','2015','2016','2017','2018','2019','2020','2021'],inplace=True)
weight.head(3)

,2014.1,2014.2,2014.3,2014.4,2014.5,2014.6,2014.7,2014.8,2014.9,2014.10,...,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11,2021.12
1,0.079836,0.071384,0.085279,0.082685,0.092022,0.091002,0.095069,0.090931,0.089713,0.082509,...,0.085501,0.090321,0.088547,0.094818,0.092978,0.089146,0.087845,0.087312,0.077059,0.067273


- 월별로 가중치에 맞게 빈 데이터 채우기

In [11]:
for k in range(perYear.shape[0]):
    for j in range(int(perYear.shape[1])): 
        # j번째 년도의 월별 가중치를 확률로 두고
        probabilities=weight.values.tolist()[0][j*12:j*12+12]
        counts = {incident: 0 for incident in range(1, 13)}
        # k번째 자치구 j번째 년도의 범죄 건수만큼 실행
        for i in range(perYear.iloc[k,j]):
            incident = random.choices(range(1, 13), probabilities)[0]
            # 월별로 선택된 건수를 카운팅
            counts[incident] += 1
        # 월별_자치구별_범죄건수 데이터 채우기
        for i in range(1,13):
            perMonth.iloc[k,j*12+i-1]=counts[i]

perMonth.head(3)

# 데이터프레임 저장
# perMonth.to_csv('./절도_월별_자치구별.csv')

,2014.1,2014.2,2014.3,2014.4,2014.5,2014.6,2014.7,2014.8,2014.9,2014.10,...,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11,2021.12
자치구,,,,,,,,,,,,,,,,,,,,,
종로구,191,150,222,194,204,201,221,198,206,167,...,89,110,93,111,98,90,88,99,91,75
중구,178,171,236,208,226,244,261,232,274,202,...,112,116,107,122,120,121,93,106,91,92
용산구,114,110,134,140,130,138,169,141,134,135,...,81,87,84,96,96,87,85,78,79,65


### 범죄종류_월별_자치구별_범죄건수  => 범죄종류_일별_자치구별_범죄건수
- 범죄종류_요일별_가중치 생성

In [64]:
perDaysOfTheWeek=pd.read_csv('../범죄발생요일.csv',header=1,index_col=0).iloc[:,:8]
perDaysOfTheWeek.replace('-',0,inplace=True)
perDaysOfTheWeek=perDaysOfTheWeek.astype('int32')
perDaysOfTheWeek

,2014,2015,2016,2017,2018,2019,2020,2021
요일별(1),,,,,,,,
합계,266784,246424,203573,184355,177458,187629,180067,166782
일,36895,33285,28812,26008,24902,25557,23840,21737
월,35699,34138,27767,25101,24469,26365,25288,23478
화,36970,33529,27700,24886,24380,25773,25256,23516
수,36689,34985,27953,25226,24841,26133,25614,23794
목,36805,33749,27903,25831,24490,26197,25051,24036
금,40471,37472,30936,27601,26498,28251,27397,25275
토,42693,39266,32502,29702,27878,29353,27621,24946
미상,562,0,0,0,0,0,0,0


In [65]:
# 2014년도 미상 존재 -> 2014년도 월~일 비율로 가중치 설정해서 배분
probabilities=perDaysOfTheWeek['2014'].tolist()[1:-1]
for i in range(len(probabilities)):
    probabilities[i]=probabilities[i]/sum(perDaysOfTheWeek['2014'].tolist()[1:-1])
counts = {incident: 0 for incident in range(1, 8)}
for i in range(perDaysOfTheWeek['2014'].tolist()[-1]):
    incident = random.choices(range(1, 8), probabilities)[0]
    counts[incident] += 1
for i in range(1,8):
    perDaysOfTheWeek.iloc[i,0]=perDaysOfTheWeek.iloc[i,0] + counts[i]
perDaysOfTheWeek.drop(index='미상',inplace=True)
perDaysOfTheWeek.columns=['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
# 요일 / 합계  -> 요일별 가중치 설정
for j in range(perDaysOfTheWeek.shape[1]):
    for i in range(perDaysOfTheWeek.shape[0]-1):
        perDaysOfTheWeek.iloc[i+1,j]=perDaysOfTheWeek.iloc[i+1,j]/perDaysOfTheWeek.iloc[0,j]
perDaysOfTheWeek=perDaysOfTheWeek.drop(index='합계')
perDaysOfTheWeek

,2014,2015,2016,2017,2018,2019,2020,2021
요일별(1),,,,,,,,
일,0.138584,0.135072,0.141532,0.141076,0.140326,0.136210,0.132395,0.130332
월,0.134082,0.138534,0.136398,0.136156,0.137886,0.140517,0.140437,0.140771
화,0.138869,0.136062,0.136069,0.134990,0.137385,0.137361,0.140259,0.140998
수,0.137842,0.141971,0.137312,0.136834,0.139982,0.139280,0.142247,0.142665
목,0.138258,0.136955,0.137066,0.140116,0.138004,0.139621,0.139120,0.144116
금,0.152011,0.152063,0.151965,0.149717,0.149320,0.150568,0.152149,0.151545
토,0.160354,0.159343,0.159658,0.161113,0.157096,0.156442,0.153393,0.149572


- 범죄종류_일별_가중치 생성

In [13]:
# 2014.1.1 ~ 2021.12.31 요일
daysOfTheWeeks=pd.read_csv('../일별요일.csv')
daysOfTheWeeks

,2014.1.1,2014.1.2,2014.1.3,2014.1.4,2014.1.5,2014.1.6,2014.1.7,2014.1.8,2014.1.9,2014.1.10,...,2021.12.22,2021.12.23,2021.12.24,2021.12.25,2021.12.26,2021.12.27,2021.12.28,2021.12.29,2021.12.30,2021.12.31
0,수,목,금,토,일,월,화,수,목,금,...,수,목,금,토,일,월,화,수,목,금


In [73]:
# 2014.1 ~ 2021.12 날짜수
dayspermonth=pd.read_csv('../월별날짜수.csv')
dayspermonth

,2014.1,2014.2,2014.3,2014.4,2014.5,2014.6,2014.7,2014.8,2014.9,2014.10,...,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11,2021.12
0,31,28,31,30,31,30,31,31,30,31,...,31,30,31,30,31,31,30,31,30,31


In [66]:
for i in range(14,22):      # i : 14~21년도
    for j in range(1,13):   # j : 1~12월
        for k in range(dayspermonth.loc[0,f'20{i}.{j}']):   # k : i년 j월의 날짜
            # 요일을 해당 년도에 맞는 가중치로 변경
            daysOfTheWeeks[f'20{i}.{j}.{k+1}']=perDaysOfTheWeek.loc[daysOfTheWeeks.loc[0,f'20{i}.{j}.{k+1}'],f'20{i}']
daysOfTheWeeks

,2014.1.1,2014.1.2,2014.1.3,2014.1.4,2014.1.5,2014.1.6,2014.1.7,2014.1.8,2014.1.9,2014.1.10,...,2021.12.22,2021.12.23,2021.12.24,2021.12.25,2021.12.26,2021.12.27,2021.12.28,2021.12.29,2021.12.30,2021.12.31
0,0.137842,0.138258,0.152011,0.160354,0.138584,0.134082,0.138869,0.137842,0.138258,0.152011,...,0.142665,0.144116,0.151545,0.149572,0.130332,0.140771,0.140998,0.142665,0.144116,0.151545


- 일별로 가중치에 맞게 데이터프레임 생성

In [75]:
# 일별 빈 데이터프레임
perDay=pd.DataFrame(index=perMonth.index, columns=daysOfTheWeeks.columns)
for idx in perDay.index:
    num=0
    for i in range(14, 22):
        for j in range(1, 13):
            # i년도 j월의 일별 가중치
            probabilities=daysOfTheWeeks.values.tolist()[0][num:num+dayspermonth.loc[0,f'20{i}.{j}']]
            num+=dayspermonth.loc[0,f'20{i}.{j}']
            counts = {incident: 0 for incident in range(1, dayspermonth.loc[0,f'20{i}.{j}']+1)}
            # i년도 j월의 범죄건수만큼 실행
            for _ in range(perMonth.loc[idx,f'20{i}.{j}']):
                incident = random.choices(range(1, dayspermonth.loc[0,f'20{i}.{j}']+1), probabilities)[0]
                counts[incident] += 1
            # i년도 j월 k일 데이터 입력
            for k in range(dayspermonth.loc[0,f'20{i}.{j}']):
                perDay.loc[idx,f'20{i}.{j}.{k+1}']=counts[k+1]

perDay.head(3)

# 데이터프레임 저장
# perDay.to_csv('./절도_일별_자치구별.csv')

,2014.1.1,2014.1.2,2014.1.3,2014.1.4,2014.1.5,2014.1.6,2014.1.7,2014.1.8,2014.1.9,2014.1.10,...,2021.12.22,2021.12.23,2021.12.24,2021.12.25,2021.12.26,2021.12.27,2021.12.28,2021.12.29,2021.12.30,2021.12.31
자치구,,,,,,,,,,,,,,,,,,,,,
종로구,8,6,7,5,13,3,6,5,7,6,...,0,3,5,1,4,1,4,1,5,2
중구,10,5,8,3,8,6,9,7,5,6,...,5,1,4,4,2,2,3,4,3,4
용산구,3,3,4,5,2,4,4,0,3,2,...,1,4,1,3,2,1,2,2,1,3


### 범죄종류_일별_자치구별_범죄건수 -> 범죄종류_일별_동별_범죄건수

In [77]:
# 동별 가중치
dong_weight=pd.read_csv('../동별_가중치.csv')
dong_weight=dong_weight.dropna(axis=0)
dong_weight=dong_weight[['구','동','가중치']]
dong_weight.reset_index(drop=True,inplace=True)
dong_weight.head(3)

,구,동,가중치
0,강남구,개포동,0.047407
1,강남구,논현동,0.164444
2,강남구,대치동,0.131852


In [78]:
# 구별 동개수
DongPerGu=dong_weight[['구','동']].groupby(by='구').count().T
DongPerGu

구,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,도봉구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
동,14,10,4,9,3,7,10,3,6,4,...,18,38,13,3,32,31,11,69,60,6


In [82]:
# 동별 가중치가 이름순이라서 똑같이 정렬
perDay.sort_index(inplace=True)
# 일별_동별 빈 데이터프레임
perDong=pd.DataFrame(index=dong_weight[['구','동']], columns=perDay.columns)
perDong.head(3)

,2014.1.1,2014.1.2,2014.1.3,2014.1.4,2014.1.5,2014.1.6,2014.1.7,2014.1.8,2014.1.9,2014.1.10,...,2021.12.22,2021.12.23,2021.12.24,2021.12.25,2021.12.26,2021.12.27,2021.12.28,2021.12.29,2021.12.30,2021.12.31
"(강남구, 개포동)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(강남구, 논현동)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(강남구, 대치동)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
# 일별_동별 데이터 채워넣기
num=0
for k in range(DongPerGu.shape[1]):
    # k번째 구의 동별 가중치
    probabilities=dong_weight.가중치.tolist()[num:num+DongPerGu.iloc[0,k]]
    for j in range(perDong.shape[1]):
        counts = {incident: 0 for incident in range(1, DongPerGu.iloc[0,k]+1)}
        for _ in range(perDay.iloc[k,j]):
            incident = random.choices(range(1, DongPerGu.iloc[0,k]+1), probabilities)[0]
            counts[incident] += 1
        for i in range(DongPerGu.iloc[0,k]):
            perDong.iloc[num+i,j]=counts[i+1]
    num+=DongPerGu.iloc[0,k]

perDong.head(3)

# 데이터프레임 저장
# perDong.to_csv('./절도_일별_동별.csv')

,2014.1.1,2014.1.2,2014.1.3,2014.1.4,2014.1.5,2014.1.6,2014.1.7,2014.1.8,2014.1.9,2014.1.10,...,2021.12.22,2021.12.23,2021.12.24,2021.12.25,2021.12.26,2021.12.27,2021.12.28,2021.12.29,2021.12.30,2021.12.31
"(강남구, 개포동)",0,0,3,0,1,0,0,0,2,0,...,0,0,0,0,0,1,0,0,0,1
"(강남구, 논현동)",0,2,1,3,2,1,3,4,0,3,...,1,2,1,1,1,0,1,1,2,1
"(강남구, 대치동)",2,3,0,0,0,0,0,1,0,3,...,0,1,1,2,0,1,0,0,0,0


### 범죄종류_월별_동별_범죄건수 데이터 생성

In [87]:
# 일별_동별 빈 데이터프레임
perDong_gu=pd.DataFrame(index=dong_weight[['구','동']], columns=perMonth.columns)
# 월별_동별 데이터 채워넣기
num=0
for k in range(DongPerGu.shape[1]):
    probabilities=dong_weight.가중치.tolist()[num:num+DongPerGu.iloc[0,k]]
    for j in range(perDong_gu.shape[1]):
        counts = {incident: 0 for incident in range(1, DongPerGu.iloc[0,k]+1)}
        for _ in range(perMonth.iloc[k,j]):
            incident = random.choices(range(1, DongPerGu.iloc[0,k]+1), probabilities)[0]
            counts[incident] += 1
        for i in range(DongPerGu.iloc[0,k]):
            perDong_gu.iloc[num+i,j]=counts[i+1]
    num+=DongPerGu.iloc[0,k]

perDong_gu.head(3)

# 데이터프레임 저장
# perDong_gu.to_csv('./절도_월별_동별.csv')

,2014.1,2014.2,2014.3,2014.4,2014.5,2014.6,2014.7,2014.8,2014.9,2014.10,...,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11,2021.12
"(강남구, 개포동)",7,9,7,12,12,10,7,10,6,10,...,3,7,2,9,9,4,4,7,8,3
"(강남구, 논현동)",32,25,37,35,33,40,29,31,36,19,...,18,21,14,18,14,13,14,14,20,10
"(강남구, 대치동)",30,23,34,29,34,20,33,22,26,29,...,13,11,8,20,20,9,12,22,10,13


### 사건별 리스트

In [4]:
cases=pd.DataFrame(index=range(perDong.sum().sum()),columns=range(8)) # rows = 절도 전체 발생 건수
cases.columns=['년','월','일','자치구','동','주소','위도','경도']
n=0
for i in range(perDong.shape[0]):
    for j in range(perDong.shape[1]):
        for _ in range(perDong.iloc[i,j]):
            cases.iloc[n,0]=perDong.columns[j].split('.')[0]
            cases.iloc[n,1]=perDong.columns[j].split('.')[1]
            cases.iloc[n,2]=perDong.columns[j].split('.')[2]
            cases.iloc[n,3]=perDong.index[i][0]
            cases.iloc[n,4]=perDong.index[i][1]
            n+=1

cases.head(3)

# 데이터프레임 저장
# cases.to_csv('절도_사건별_리스트.csv')

,년,월,일,자치구,동,주소,위도,경도
0,2014,1,4,강남구,개포동,None,NaN,NaN
1,2014,1,4,강남구,개포동,None,NaN,NaN
2,2014,1,7,강남구,개포동,None,NaN,NaN


In [258]:
# 행정동 법정동 달라서 차이나는거 수정
for i in cases[cases.동=='상일제1동'].index:
    cases.iloc[i,4]='상일동'
for i in cases[cases.동=='공릉2동'].index:
    cases.iloc[i,4]='공릉동'
for i in cases[cases.동=='왕십리로10길'].index:
    cases.iloc[i,4]='성수동1가'
for i in cases[cases.동=='내자동'].index:
    cases.iloc[i,4]='사직동'
for i in cases[cases.동=='예지동'].index:
    cases.iloc[i,4]='종로4가'
for i in cases[cases.동=='산림동'].index:
    cases.iloc[i,4]=f'을지로{random.choice([3,4,5])}가'

- 해당 동별 랜덤한 주소, 좌표 부여

In [264]:
locations=pd.read_csv('../최종위경도.csv',index_col=0)
locations=locations.sort_values(by=['시군구','법정동'])
locations.drop(index=locations[locations.법정동.isna()].index,inplace=True)
locations.drop(index=locations[locations.위도==0].index,inplace=True)
locations.drop(index=locations[locations.경도==0].index,inplace=True)
locations.reset_index(drop=True,inplace=True)
locations.head(3)

,시군구,법정동,주소,위도,경도
0,강남구,개포동,서울특별시 강남구 개포로 303,37.482682,127.055556
1,강남구,개포동,서울특별시 강남구 개포로109길 9,37.496894,127.077245
2,강남구,개포동,서울특별시 강남구 개포로 516,37.489134,127.070891


In [204]:
for i in tqdm(range(cases.shape[0])):
    if len(locations[locations.법정동==cases.iloc[i,4]].index) !=0:
        idx=random.choice(locations[locations.법정동==cases.iloc[i,4]].index)
        cases.iloc[i,5]=locations.iloc[idx,2]
        cases.iloc[i,6]=locations.iloc[idx,3]
        cases.iloc[i,7]=locations.iloc[idx,4]

cases.head(3)

  0%|          | 0/355922 [00:00<?, ?it/s]

,년,월,일,자치구,동,주소,위도,경도
0,2014,1,4,강남구,개포동,서울특별시 강남구 선릉로10길 10-8,37.483520,127.062304
1,2014,1,4,강남구,개포동,서울특별시 강남구 개포로 516,37.489134,127.070891
2,2014,1,7,강남구,개포동,서울특별시 강남구 개포로35길 13,37.479962,127.049926


In [268]:
cases.isna().sum()

년      0
월      0
일      0
자치구    0
동      0
주소     0
위도     0
경도     0
dtype: int64

In [269]:
# 데이터프레임 저장
# cases.to_csv('./절도_사건별_리스트(주소추가).csv')